In [ ]:
import React, { useState, useEffect } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery, useMutation, useQueryClient } from '@tanstack/react-query';
import { motion, AnimatePresence } from 'framer-motion';
import { format } from 'date-fns';
import { toast } from 'sonner';
import { ArrowLeft, Utensils, Tag, X } from 'lucide-react';
import { Button } from "@/components/ui/button";
import {
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
} from "@/components/ui/select";
import { Badge } from "@/components/ui/badge";

import CameraScanner from '../components/scanner/CameraScanner';
import ScanResult from '../components/scanner/ScanResult';
import OnboardingFlow from '../components/onboarding/OnboardingFlow';

export default function ScanPage() {
  const [isProcessing, setIsProcessing] = useState(false);
  const [scanResult, setScanResult] = useState(null);
  const [mealType, setMealType] = useState('snack');
  const [selectedTags, setSelectedTags] = useState([]);
  const [uploadedImageUrl, setUploadedImageUrl] = useState(null);

  const availableTags = [
    'Homemade', 'Restaurant', 'Takeout', 'Meal Prep',
    'Cheat Meal', 'Pre-Workout', 'Post-Workout', 'Late Night',
    'Comfort Food', 'Healthy Choice', 'Organic', 'Fast Food'
  ];
  
  const queryClient = useQueryClient();

  const { data: userProfiles, isLoading: profileLoading } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles?.[0];
  const needsOnboarding = !profileLoading && (!userProfile || !userProfile.onboarding_complete);

  const createProfileMutation = useMutation({
    mutationFn: (data) => base44.entities.UserProfile.create(data),
    onSuccess: () => {
      queryClient.invalidateQueries({ queryKey: ['userProfile'] });
    },
  });

  const createScanMutation = useMutation({
    mutationFn: (data) => base44.entities.FoodScan.create(data),
    onSuccess: () => {
      queryClient.invalidateQueries({ queryKey: ['foodScans'] });
      toast.success('Meal added to your log!');
      setScanResult(null);
      setUploadedImageUrl(null);
    },
  });

  const handleOnboardingComplete = async (profileData) => {
    await createProfileMutation.mutateAsync(profileData);
  };

  const handleCapture = async (file) => {
    setIsProcessing(true);
    setScanResult(null);

    try {
      // Upload image first
      const { file_url } = await base44.integrations.Core.UploadFile({ file });
      setUploadedImageUrl(file_url);

      // Analyze with AI
      const analysisResult = await base44.integrations.Core.InvokeLLM({
        prompt: `Analyze this food image and provide detailed nutritional information.
        
User profile:
- Dietary goal: ${userProfile?.dietary_goal || 'general health'}
- Diet type: ${userProfile?.diet_type || 'none'}
- Allergies: ${userProfile?.allergies?.join(', ') || 'none'}

Please identify the food and provide:
1. Food name and category
2. Estimated portion size
3. Calories and macronutrients (protein, carbs, fats, fiber, sugar, sodium)
4. Health score (1-10) based on the user's dietary goal
5. List of potential allergens
6. 2-3 personalized suggestions for healthier choices or tips

Be specific and accurate with nutritional estimates based on typical serving sizes.`,
        file_urls: [file_url],
        response_json_schema: {
          type: 'object',
          properties: {
            food_name: { type: 'string' },
            food_category: { type: 'string' },
            portion_size: { type: 'string' },
            calories: { type: 'number' },
            protein: { type: 'number' },
            carbs: { type: 'number' },
            fats: { type: 'number' },
            fiber: { type: 'number' },
            sugar: { type: 'number' },
            sodium: { type: 'number' },
            health_score: { type: 'number' },
            allergens: { type: 'array', items: { type: 'string' } },
            suggestions: { type: 'array', items: { type: 'string' } },
          },
          required: ['food_name', 'calories', 'protein', 'carbs', 'fats']
        }
      });

      setScanResult({
        ...analysisResult,
        image_url: file_url,
      });
    } catch (error) {
      console.error('Scan failed:', error);
      toast.error('Failed to analyze food. Please try again.');
    } finally {
      setIsProcessing(false);
    }
  };

  const handleAddToLog = async () => {
    if (!scanResult) return;

    await createScanMutation.mutateAsync({
      ...scanResult,
      meal_type: mealType,
      tags: selectedTags,
      scan_date: format(new Date(), 'yyyy-MM-dd'),
    });
  };

  const handleScanAgain = () => {
    setScanResult(null);
    setUploadedImageUrl(null);
    setSelectedTags([]);
  };

  const toggleTag = (tag) => {
    setSelectedTags(prev => 
      prev.includes(tag) 
        ? prev.filter(t => t !== tag)
        : [...prev, tag]
    );
  };

  if (profileLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  if (needsOnboarding) {
    return <OnboardingFlow onComplete={handleOnboardingComplete} />;
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30">
      <div className="max-w-lg mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="flex items-center justify-between mb-8"
        >
          <div>
            <h1 className="text-2xl font-bold text-slate-800">Scan Food</h1>
            <p className="text-slate-500 text-sm">Point your camera at any meal</p>
          </div>
          {scanResult && (
            <Button
              variant="ghost"
              size="icon"
              onClick={handleScanAgain}
              className="rounded-full"
            >
              <ArrowLeft className="w-5 h-5" />
            </Button>
          )}
        </motion.div>

        {/* Meal Type Selector */}
        {!scanResult && (
          <motion.div 
            initial={{ opacity: 0 }}
            animate={{ opacity: 1 }}
            className="space-y-4 mb-6"
          >
            <Select value={mealType} onValueChange={setMealType}>
              <SelectTrigger className="w-full h-12 rounded-2xl bg-white border-slate-200">
                <div className="flex items-center gap-2">
                  <Utensils className="w-4 h-4 text-slate-400" />
                  <SelectValue placeholder="Meal type" />
                </div>
              </SelectTrigger>
              <SelectContent>
                <SelectItem value="breakfast">🌅 Breakfast</SelectItem>
                <SelectItem value="lunch">☀️ Lunch</SelectItem>
                <SelectItem value="dinner">🌙 Dinner</SelectItem>
                <SelectItem value="snack">🍎 Snack</SelectItem>
              </SelectContent>
            </Select>

            {/* Tags Selector */}
            <div className="bg-white rounded-2xl p-4 border border-slate-200">
              <div className="flex items-center gap-2 mb-3">
                <Tag className="w-4 h-4 text-slate-400" />
                <span className="text-sm font-medium text-slate-600">Add Tags (Optional)</span>
              </div>
              <div className="flex flex-wrap gap-2">
                {availableTags.map(tag => {
                  const isSelected = selectedTags.includes(tag);
                  return (
                    <Badge
                      key={tag}
                      variant={isSelected ? "default" : "outline"}
                      className={`cursor-pointer transition-all ${
                        isSelected 
                          ? 'bg-emerald-500 hover:bg-emerald-600 text-white' 
                          : 'hover:border-emerald-300'
                      }`}
                      onClick={() => toggleTag(tag)}
                    >
                      {tag}
                      {isSelected && <X className="w-3 h-3 ml-1" />}
                    </Badge>
                  );
                })}
              </div>
            </div>
          </motion.div>
        )}

        {/* Scanner or Result */}
        <AnimatePresence mode="wait">
          {scanResult ? (
            <motion.div
              key="result"
              initial={{ opacity: 0, scale: 0.95 }}
              animate={{ opacity: 1, scale: 1 }}
              exit={{ opacity: 0, scale: 0.95 }}
            >
              <ScanResult
                result={scanResult}
                onAddToLog={handleAddToLog}
                onScanAgain={handleScanAgain}
                userAllergies={userProfile?.allergies || []}
              />
            </motion.div>
          ) : (
            <motion.div
              key="scanner"
              initial={{ opacity: 0 }}
              animate={{ opacity: 1 }}
              exit={{ opacity: 0 }}
              className="bg-white rounded-3xl p-8 shadow-sm border border-slate-100"
            >
              <CameraScanner
                onCapture={handleCapture}
                isProcessing={isProcessing}
              />
            </motion.div>
          )}
        </AnimatePresence>

        {/* Tips */}
        {!scanResult && !isProcessing && (
          <motion.div 
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.3 }}
            className="mt-8 space-y-3"
          >
            <p className="text-sm font-medium text-slate-600 mb-3">📸 Scanning tips</p>
            <div className="grid gap-2 text-sm text-slate-500">
              <div className="flex items-start gap-2">
                <span className="w-5 h-5 rounded-full bg-emerald-100 text-emerald-600 flex items-center justify-center text-xs flex-shrink-0">1</span>
                <span>Ensure good lighting for accurate results</span>
              </div>
              <div className="flex items-start gap-2">
                <span className="w-5 h-5 rounded-full bg-emerald-100 text-emerald-600 flex items-center justify-center text-xs flex-shrink-0">2</span>
                <span>Capture the entire plate or food item</span>
              </div>
              <div className="flex items-start gap-2">
                <span className="w-5 h-5 rounded-full bg-emerald-100 text-emerald-600 flex items-center justify-center text-xs flex-shrink-0">3</span>
                <span>Works with meals, snacks, and packaged foods</span>
              </div>
            </div>
          </motion.div>
        )}
      </div>
    </div>
  );
}